# Deep learning of rewards

This illustrates the **DeepRewardController**. This controller uses a neural network to attempt to predict the action with the highest reward value. In other words, a perfectly-trained controller would be identical to the **RewardController**.

In [1]:
from pod.board import PodBoard
from pod.ai.deep_reward_controller import DeepRewardController

board = PodBoard()
controller = DeepRewardController(board)

### Training

First, we create some training data: a bunch of pods in various states around the target checkpoint.

In [2]:
from pod.ai.ai_utils import gen_pods, MAX_VEL
from pod.constants import Constants
import math
import numpy as np

pods_everywhere = gen_pods(
    board.checkpoints[0],
    np.arange(Constants.check_radius(), 10000, 1000),
    np.arange(math.pi * -0.9, math.pi * 0.91, math.pi * 0.2),
    np.arange(math.pi * -0.9, math.pi * 0.91, math.pi * 0.2),
    np.arange(0, MAX_VEL + 1, MAX_VEL / 5)
)

# TODO: training goes much better if I add extra pods pointing towards the check...why?
pods_focused = gen_pods(
    board.checkpoints[0],
    np.arange(Constants.check_radius(), 10000, 1000),
    np.arange(-0.3, 0.3, 0.05),
    np.arange(math.pi * -0.9, math.pi * 0.91, math.pi * 0.2),
    np.arange(0, MAX_VEL + 1, MAX_VEL / 5)
)

pods = [*pods_everywhere, *pods_focused]

print("{} total states".format(len(pods)))

60000 pods generated
72000 pods generated
132000 total states


Now that we have a bunch of pod states, we can perform the training. The labels (i.e. the target output for each state) are calculated as whatever produces the highest reward.

In [ ]:
import matplotlib.pyplot as plt

history = controller.train(pods, 50)

plt.plot(history.history['accuracy'])
#plt.plot(history.history['loss'])
plt.legend([
    "Accuracy",
#    "Loss"
])
plt.show()

Epoch 1/50
4125/4125 [==============================] - 3s 570us/step - loss: 1.4657 - accuracy: 0.3035
Epoch 2/50
4125/4125 [==============================] - 2s 577us/step - loss: 0.9748 - accuracy: 0.5835
Epoch 3/50
4125/4125 [==============================] - 2s 567us/step - loss: 0.4714 - accuracy: 0.92040s - l
Epoch 4/50
4125/4125 [==============================] - 2s 558us/step - loss: 0.3097 - accuracy: 0.9420
Epoch 5/50
4125/4125 [==============================] - 2s 571us/step - loss: 0.2367 - accuracy: 0.9465
Epoch 6/50
4125/4125 [==============================] - 2s 602us/step - loss: 0.1972 - accuracy: 0.9547
Epoch 7/50
4125/4125 [==============================] - 2s 583us/step - loss: 0.1737 - accuracy: 0.9628
Epoch 8/50
4125/4125 [==============================] - 2s 561us/step - loss: 0.1566 - accuracy: 0.9672
Epoch 9/50
4125/4125 [==============================] - 2s 551us/step - loss: 0.1434 - accuracy: 0.9711
Epoch 10/50
4125/4125 [==============================] - 2

### Play

Now that the model has been trained, let's see what it can do!

As a comparison, we also add a **SimpleController** (which simply goes full-speed toward the next checkpoint) and **RewardController** (which takes whatever action produces the highest reward).

In [ ]:
TURNS = 200

from pod.util import PodState
from pod.game import Player
from pod.drawer import Drawer
from IPython.display import Image
from pod.ai.reward_controller import RewardController
from pod.controller import SimpleController

deep_player = Player(controller)
base_player = Player(RewardController(board))
simple_player = Player(SimpleController())

drawer = Drawer(board, [deep_player, base_player, simple_player])

deep_player.reset(board)
base_player.reset(board)
simple_player.reset(board)

file = '/tf/notebooks/pods.gif'
drawer.animate(file, TURNS)
Image(filename = file)

The following shows the rewards for the two players in the above run.

In [ ]:
deep_player.reset(board)
base_player.reset(board)
simple_player.reset(board)

drawer.chart_rewards(TURNS)